<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Setup" data-toc-modified-id="Setup-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Setup</a></span></li><li><span><a href="#Utilities" data-toc-modified-id="Utilities-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Utilities</a></span></li></ul></div>

## Setup

In [1]:
%cd ~/src/Sunrise-Demo/
from __future__ import annotations
try:
#     raise ImportError(r"Force re-install library")
    from mediocreatbest import auto, run
    
except ImportError:
    %pip install --quiet --upgrade pip
    %pip install --upgrade --force-reinstall \
        mediocreatbest@git+https://gist.github.com/player1537/3457b026ed6ef6696d758517f55a58df.git
    from mediocreatbest import auto, run
    

/home/thobson2/src/Sunrise-Demo


In [2]:
try:
    import ospray
except ImportError:
    %pip install --user \
        ospray@git+https://gist.github.com/player1537/c06faa784cc993fd6fd9c112d9feb5d9.git
    import ospray
    

In [3]:
%%scope
version = '2.12.0'
url = f'https://github.com/ospray/ospray/releases/download/v{version}/ospray-{version}.x86_64.linux.tar.gz'
archive = auto.xdg_base_dirs.xdg_cache_home() / f'ospray-{version}.tar.gz'
prefix = auto.xdg_base_dirs.xdg_cache_home() / f'ospray-{version}'
library = prefix / 'lib' / 'libospray.so'

def Download():
    if archive.exists():
        print(f"""Already downloaded {archive}""")
        return
    
    print(f"""Download {url} to {archive}""")
    auto.urllib.request.urlretrieve(
        url,
        filename=archive,
    )

def Extract():
    if prefix.exists():
        print(f"Directory {prefix} already exists""")
        return
    
    print(f"""Extract {archive} to {prefix}""")
    with auto.tempfile.TemporaryDirectory() as tmp:
        tmp = auto.pathlib.Path(tmp)
        
        with auto.tarfile.open(archive) as tar:
            tar.extractall(tmp)
        
        auto.shutil.move(
            tmp / f'ospray-{version}.x86_64.linux',
            prefix,
        )
        
def Library() -> 'lib':
    return ospray.load_library(library)
    
# auto.os.environ['LD_LIBRARY_PATH'] = \
#     str(auto.pathlib.Path.home() / 'opt' / '')

global lib

try:
    lib
    print(f"""Already initialized OSPRay""")

except NameError:
    try:
        lib = Library()
    except OSError:
        Download()
        Extract()
        lib = Library()
    
    print(f"""Initializing OSPRay""")
    lib.ospInit(None, None)
    

Initializing OSPRay


## Utilities

In [4]:
def Data(
    array: auto.numpy.ndarray,
    /,
    *,
    type: lib.OSPDataType,
    share: bool=False,
) -> lib.OSPData:
    if len(array.shape) == 0:
        array = array[None, None, None]

    elif len(array.shape) == 1:
        array = array[:, None, None]

    elif len(array.shape) == 2:
        array = array[:, :, None]

    elif len(array.shape) == 3:
        array = array[:, :, :]

    else:
        raise NotImplementedError()

    src = lib.ospNewSharedData(
        array.ctypes.data, type,
        array.shape[0], array.strides[0],
        array.shape[1], array.strides[1],
        array.shape[2], array.strides[2],
    )
    lib.ospCommit(src)
    
    if share:
        return src

    dst = lib.ospNewData(type, *array.shape)
    lib.ospCopyData(src, dst, 0, 0, 0)
    lib.ospCommit(dst)

    lib.ospRelease(src)
    return dst


In [5]:
def Read(
    path: auto.pathlib.Path,
    /,
    *,
    dtype: auto.numpy.DType,
    mmap: bool=True,
) -> auto.numpy.NDArray:
    path = auto.pathlib.Path(path)

    print(f"""Reading {path.stat().st_size :,d} bytes from {path}""")
    with open(path, 'rb') as f:
        def Read(fmt: str, /) -> tuple:
            size = auto.struct.calcsize(fmt)
            data = f.read(size)
            assert len(data) == size
            return auto.struct.unpack(fmt, data)
        
        N ,= Read('I')
        shape = Read(f'{N}I')
        
        if mmap:
            data = auto.numpy.memmap(
                f,
                mode='r',
                dtype=dtype,
                offset=f.tell(),
                shape=shape,
            )
        
        else:
            data = auto.numpy.fromfile(f, dtype=dtype)
            data = data.reshape(shape)

    return data


In [6]:
def DataRead(
    path: auto.pathlib.Path,
    /,
    dtype: auto.numpy.DType,
    type: lib.OSPDataType,
    share: bool=False,
    mmap: bool=True,
) -> lib.OSPData:
    return Data(
        Read(
            path,
            dtype=dtype,
        ),
        type=type,
        share=share,
        mmap=mmap,
    )


In [7]:
# try:
#     __scene
# except NameError:
#     __scene = {}

# @auto.mediocreatbest.immediate
# class S:
#     def __setitem__()
    
try:
    S
except NameError:
    S = {}


In [35]:
%%scope --skip -n Pink -i i -o pink
S[f"pink{i}texture/data:vec3f[]"] = Read(
    f"data/pink{i}/OSPTexture.texture2d.data.vec3f.bin",
    dtype=[ ('r', 'f4'), ('g', 'f4'), ('b', 'f4') ],
)

S[f"pink{i}/texture/data"] = Data(
    S[f"pink{i}/texture/data:vec3f[]"],
    type=lib.OSP_VEC3F,
    share=True,
)

S[f"pink{i}/texture"] = lib.ospNewTexture(b'texture2d')
lib.ospSetObject(S[f"pink{i}/texture"],
    b'data', S[f"pink{i}/texture/data"])
lib.ospCommit(S[f"pink{i}/texture"])

S[f"pink{i}/material"] = lib.ospNewMaterial(None, b'obj')
lib.ospSetObject(S[f"pink{i}/material"],
    b'map_kd', S[f"pink{i}/texture"])
lib.ospCommit(S[f"pink{i}/material"])

pink = S[f"pink{i}/material"]


In [36]:
pink0 = Pink(i=0)
pink1 = Pink(i=1)
pink2 = Pink(i=2)
pink3 = Pink(i=3)


Reading 402,653,196 bytes from data/pink0/OSPTexture.texture2d.data.vec3f.bin
Reading 402,653,196 bytes from data/pink1/OSPTexture.texture2d.data.vec3f.bin
Reading 402,653,196 bytes from data/pink2/OSPTexture.texture2d.data.vec3f.bin
Reading 402,653,196 bytes from data/pink3/OSPTexture.texture2d.data.vec3f.bin


In [37]:
%%scope -n Park -o park
S["park/OSPGeometry:mesh:vertex.position:vec3f[]"] = Read(
    'data/park/OSPGeometry.mesh.vertex.position.vec3f.bin',
    dtype=[ ('x', 'f4'), ('y', 'f4'), ('z', 'f4') ],
)

S["park/OSPGeometry:mesh:vertex.texcoord:vec2f[]"] = Read(
    'data/park/OSPGeometry.mesh.vertex.texcoord.vec2f.bin',
    dtype=[ ('u', 'f4'), ('v', 'f4') ],
)

S["park/OSPGeometry:mesh:index:vec4ui[]"] = Read(
    'data/park/OSPGeometry.mesh.index.vec4ui.bin',
    dtype=[ ('a', 'u4'), ('b', 'u4'), ('c', 'u4'), ('d', 'u4') ],
)

S["park/OSPGeometry:mesh:vertex.position"] = Data(
    S["park/OSPGeometry:mesh:vertex.position:vec3f[]"],
    type=lib.OSP_VEC3F,
    share=True,
)

S["park/OSPGeometry:mesh:vertex.texcoord"] = Data(
    S["park/OSPGeometry:mesh:vertex.texcoord:vec2f[]"],
    type=lib.OSP_VEC2F,
    share=True,
)

S["park/OSPGeometry:mesh:index"] = Data(
    S["park/OSPGeometry:mesh:index:vec4ui[]"],
    type=lib.OSP_VEC4UI,
    share=True,
)

S["park/OSPGeometry:mesh"] = lib.ospNewGeometry(b'mesh')
lib.ospSetObject(S["park/OSPGeometry:mesh"],
    b'vertex.position', S["park/OSPGeometry:mesh:vertex.position"])
lib.ospSetObject(S["park/OSPGeometry:mesh"],
    b'vertex.texcoord', S["park/OSPGeometry:mesh:vertex.texcoord"])
lib.ospSetObject(S["park/OSPGeometry:mesh"],
    b'index', S["park/OSPGeometry:mesh:index"])
lib.ospCommit(S["park/OSPGeometry:mesh"])

park = S["park/OSPGeometry:mesh"]


Reading 2,906,253,336 bytes from data/park/OSPGeometry.mesh.vertex.position.vec3f.bin
Reading 1,937,502,228 bytes from data/park/OSPGeometry.mesh.vertex.texcoord.vec2f.bin
Reading 3,874,472,012 bytes from data/park/OSPGeometry.mesh.index.vec4ui.bin


In [ ]:
S["park/OSPGeometricModel:geometry:OSPGeometry[]"] = auto.numpy.array([
    
])